El Tox21 data Challenge va ser un concurs realitzat al 2014. El concurs consistia en utilitzar el dataset Tox21 per fer prediccions sobre la toxicitat de certes substàncies basant-nos en la seva estructura molecular, concretament el dataset ens proporciona una sèrie de molècules en les que se'ns indica la presència (o no) de 12 terminacions tòxiques, una identificació unica per a cada molècuula i la seva representació en format SMILE,
la tasca serà predir la presència o no d'aquestes terminacions tòxiques basant-nos en la estructura molecular. En aquesta Notebook aprofitarem el fet que si pensem els àtoms com a nodes i els enllaços com a vertex, podem transformar una molècula en un graf.

In [1]:
import warnings
warnings.filterwarnings("ignore")

Com ja hem dit treballarem amb el dataset Tox21, per fer-ho s'ha implementat una classe `DataLoader` que ens permet carregar el dataset i fer la neteja inicial del mateix:
 * Eliminació de duplicats: Com que el dataset esta creat de forma experimental hi han molècules duplicades, per tant es fa una eliminació dels duplicats fent un merge de les columnes, d'aquesta forma aconseguim també reduir alguns NaNs ja que entre les columnes duplicades s'autocompleta la informació.
 * Neteja dels números de la massa molecular: La massa molecular bé en format $FW : M\;(m_1 + m_2 + \dots + m_n)$ ón $M$ és la massa total de la molècula i les $m_i$ son la descomposició de les masses individuals dels àtoms que la conformen, s'ha eliminat la descomposició ja que no és necessària.
 * S'ha fet un canvi de les columnes objectiu a float

In [2]:
from DataLoader import DataLoader

DataLoader = DataLoader()
data = DataLoader.get_processed_source()

[18:57:10] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 10
[18:57:10] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[18:57:10] ERROR: Could not sanitize molecule ending on line 21572
[18:57:10] ERROR: Explicit valence for atom # 3 Cl, 2, is greater than permitted
[18:57:11] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[18:57:11] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 1 ignored.
[18:57:11] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[18:57:11] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[18:57:11] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 10
[18:57:12] Explicit valence for atom # 2 Si, 8, is greater than permitted
[18:57:12] ERROR: Could not sanitize molecule ending on line 346021
[18:57:12] ERROR: Explicit valence for atom # 

,Formula,FW,DSSTox_CID,SR-HSE,ID,SMILES,Molecule,NR-AR,SR-ARE,NR-Aromatase,NR-ER-LBD,NR-AhR,SR-MMP,NR-ER,NR-PPAR-gamma,SR-p53,SR-ATAD5,NR-AR-LBD
0,C27H25ClN6,468.9806,25848,0.0,NCGC00178831-03,C[n+]1c2cc(N)ccc2cc2ccc(N)cc21.Nc1ccc2cc3ccc(N...,<rdkit.Chem.rdchem.Mol object at 0x0000022457E...,0.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN
1,C20H6Br4Na2O5,691.8542,5234,0.0,NCGC00166114-03,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,<rdkit.Chem.rdchem.Mol object at 0x0000022457E...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN
2,C47H83NO17,934.1584,28909,0.0,NCGC00263563-01,CO[C@@H]1[C@@H](OC)[C@H](C)[C@@](O)(CC(=O)[O-]...,<rdkit.Chem.rdchem.Mol object at 0x0000022457E...,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0
3,C52H54N4O12,927.0048,5513,1.0,NCGC00013058-02,CN(C)c1ccc(C(=C2C=CC(=[N+](C)C)C=C2)c2ccccc2)c...,<rdkit.Chem.rdchem.Mol object at 0x0000022457E...,0.0,NaN,NaN,0.0,1.0,1.0,NaN,NaN,NaN,0.0,NaN
4,C66H87N17O14,1342.5025,26683,NaN,NCGC00167516-01,CC(=O)O.CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=...,<rdkit.Chem.rdchem.Mol object at 0x00000224583...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11755,C12H22S2,230.4331,27628,0.0,NCGC00254435-01,C1CCC(SSC2CCCCC2)CC1,<rdkit.Chem.rdchem.Mol object at 0x00000224645...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11756,C7H5Cl2NS,206.0923,21783,0.0,NCGC00255499-01,NC(=S)c1c(Cl)cccc1Cl,<rdkit.Chem.rdchem.Mol object at 0x00000224645...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11758,CS2,76.1407,3947,0.0,NCGC00258720-01,S=C=S,<rdkit.Chem.rdchem.Mol object at 0x00000224645...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11760,C3H6N2S,102.1581,601,0.0,NCGC00258846-01,S=C1NCCN1,<rdkit.Chem.rdchem.Mol object at 0x00000224645...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
